### Notebook 1: Data Cleaning for train and test data

[Cleaning: Date & Time](#clean_date_time)

[Cleaning: Aggregating Number of Mosquitos](#agg)

[Cleaning: Dummy Species, Month, Carrier](#dummy) 

[Cleaning: Assigning Traps to closest Station](#ass)
    
[Cleaning Test Data](#test)

[Merging Weather data into Train and Test](#merge)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pandas_profiling import ProfileReport
from datetime import datetime
import calendar
import math

from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
train = pd.read_csv('../data/train.csv') #10506 records by 12 columns, 8610 records by 27 columns
test = pd.read_csv('../data/test.csv') #116293 records by 11 columns (extra columns: Id, missing columns: WnvPresent, NumMosquitos)

pd.set_option('display.max_columns',500)

<a id='clean_date_time'></a>

### Cleaning Train Data

In [3]:
# formating date column
train['Date'] = pd.to_datetime(train['Date'])
train['week']=  train['Date'].dt.week
train['month']=train['Date'].dt.month
train['month'] = train['month'].apply(lambda x: calendar.month_abbr[x]) #converting month number to month name

train['year']=train['Date'].dt.year
train['year_month']=[(i.year,i.month) for i in train['Date']]

<a id='agg'></a>

### Aggregating Number of Mosquitos 

Initial records will max out once there are 50 mosquitos. We need to aggregate the records to reflect total number of mosquitos captured per day.

In [4]:
nonmosquitos = [col for col in train if col != 'NumMosquitos']
train2 = train.groupby(by = nonmosquitos).sum()
train = train2.reset_index()

In [5]:
train.loc[train['NumMosquitos']>=50].sort_values(by = 'Trap') #checking number of mosquitos is aggregated properly

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,WnvPresent,week,month,year,year_month,NumMosquitos
3657,2009-07-17,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,0,29,Jul,2009,"(2009, 7)",96
6970,2013-07-08,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,0,28,Jul,2013,"(2013, 7)",192
6969,2013-07-08,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,0,28,Jul,2013,"(2013, 7)",54
3764,2009-07-24,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,30,Jul,2009,"(2009, 7)",50
3508,2009-07-10,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,0,28,Jul,2009,"(2009, 7)",129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3964,2009-07-31,"Ohare Court, Bensenville, IL 60106, USA",CULEX PIPIENS/RESTUANS,10,W OHARE,T903,"1000 W OHARE, Chicago, IL",41.957799,-87.930995,5,0,31,Jul,2009,"(2009, 7)",60
4254,2009-08-25,"Ohare Court, Bensenville, IL 60106, USA",CULEX PIPIENS/RESTUANS,10,W OHARE,T903,"1000 W OHARE, Chicago, IL",41.957799,-87.930995,5,1,35,Aug,2009,"(2009, 8)",50
5467,2011-07-25,"Ohare Court, Bensenville, IL 60106, USA",CULEX PIPIENS/RESTUANS,10,W OHARE,T903,"1000 W OHARE, Chicago, IL",41.957799,-87.930995,5,0,30,Jul,2011,"(2011, 7)",128
4083,2009-08-07,"Ohare Court, Bensenville, IL 60106, USA",CULEX PIPIENS/RESTUANS,10,W OHARE,T903,"1000 W OHARE, Chicago, IL",41.957799,-87.930995,5,0,32,Aug,2009,"(2009, 8)",87


<a id='ass'></a>

## Assigning traps to closest weather station
In order to get relevant weather data for a trap, we assign a trap to the closest of 2 available weather stations in Chicago.

In [6]:
#creating the function to find the nearest station for each trap
def nearest_station(lat, long):
    station_1_lat = 41.995
    station_1_lon = -87.933
    station_2_lat = 41.786
    station_2_lon = -87.752
    # Convert latitude and longitude to spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0
    # phi = 90 - latitude
    phi = (90.0 - lat)*degrees_to_radians
    phi1 = (90.0 - station_1_lat)*degrees_to_radians
    phi2 = (90.0 - station_2_lat)*degrees_to_radians
    # theta = longitude
    theta = long*degrees_to_radians
    theta1 = station_1_lon*degrees_to_radians
    theta2 = station_2_lon*degrees_to_radians
    # Compute spherical distance from spherical coordinates.
    cos = (math.sin(phi)*math.sin(phi1)*math.cos(theta - theta1) + math.cos(phi)*math.cos(phi1))
    arc_1 = math.acos( cos )
    cos = (math.sin(phi)*math.sin(phi2)*math.cos(theta - theta2) + math.cos(phi)*math.cos(phi2))
    arc_2 = math.acos( cos )
    if arc_1<arc_2:
        return 1
    else:
        return 2
#applying it for each trap
train['station'] = [nearest_station(train['Latitude'][i],train['Longitude'][i]) for i in range(len(train['Latitude']))]

<a id='dummy'></a>

### Dummy Variables for Mosquito Species, Month & Carrier of Virus

In [7]:
carrier = ['CULEX PIPIENS/RESTUANS', 'CULEX RESTUANS', 'CULEX PIPIENS'] # 2 mosquito species are carriers of the virus
train['carrier'] = [1 if i in carrier else 0 for i in train['Species']]

In [8]:
train = pd.get_dummies(train,columns=['month'],drop_first=True) # Aug was dropped
train = pd.get_dummies(train,columns=['Species'],drop_first=True) # species_erraticus was dropped

In [9]:
# renaming columns
moz_dummy={'Species_CULEX PIPIENS':'species_pipiens','Species_CULEX PIPIENS/RESTUANS':'species_pipiens/restuans','Species_CULEX RESTUANS':'species_restuans','Species_CULEX SALINARIUS':'species_salinarius','Species_CULEX TARSALIS':'species_tarsalis','Species_CULEX TERRITANS':'species_territans'}
months = {'month_Jul' : 'jul','month_Jun' : 'jun', 'month_May' : 'may', 'month_Oct' : 'oct', 'month_Sep' : 'sep'}
train.rename(columns=moz_dummy,inplace=True)
train.rename(columns=months,inplace=True)

In [10]:
# dropping the Address, Street, Block and AddressNumberAndStreet column as they will be difficult for hte model to make sense of
train.drop(columns  = ['Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy'], inplace = True)

In [11]:
train.shape

(8610, 22)

<a id='test'></a>

### Cleaning Test Data
Performing all the steps as for train data

In [12]:
# formating date column
test['Date'] = pd.to_datetime(test['Date'])
test['week']=  test['Date'].dt.week
test['month']=test['Date'].dt.month
test['month'] = test['month'].apply(lambda x: calendar.month_abbr[x]) #converting month number to month name

test['year']=test['Date'].dt.year
test['year_month']=[(i.year,i.month) for i in test['Date']]

In [13]:
# Assigning closest weather station to test records
test['station'] = [nearest_station(test['Latitude'][i],test['Longitude'][i]) for i in range(len(test['Latitude']))]

In [14]:
# dummy variables: if mosquito is a carrier, month, and species
carrier = ['CULEX PIPIENS/RESTUANS', 'CULEX RESTUANS', 'CULEX PIPIENS'] # 2 mosquito species are carriers of the virus
test['carrier'] = [1 if i in carrier else 0 for i in test['Species']]

test = pd.get_dummies(test,columns=['month'],drop_first=True)
test = pd.get_dummies(test,columns=['Species'],drop_first=True)

In [15]:
test.drop(columns  = ['Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy'], inplace = True)

In [16]:
test.rename(columns=moz_dummy,inplace=True)
test.rename(columns=months,inplace=True)
test.shape

(116293, 21)

### Ensuring test and train are the same shape

In [17]:
# what are the extra test columns not in training set
extra_test_col = [col for col in test.columns if col not in set(train.columns)]
print(f' extra test columns are {extra_test_col}, \nshape of test set is {test.shape}')


 extra test columns are ['Id', 'Species_UNSPECIFIED CULEX'], 
shape of test set is (116293, 21)


In [18]:
# add 'unspecified culex' column into training set  
array_zeros = np.zeros((train.shape[0], 1))
df_zeros = pd.DataFrame(array_zeros, columns=['species_unspecified_culex'])
final_train = train.join(df_zeros, how = 'left')

# add 'may' column into test set
test_array_zeros = np.zeros((test.shape[0], 1))
test_df_zeros = pd.DataFrame(test_array_zeros, columns=['may'])
final_test = test.join(test_df_zeros, how = 'left')

In [19]:
# checking the extra columns in final train vs final test
culex_rename = {'Species_UNSPECIFIED CULEX' : 'species_unspecified_culex'}
final_test.rename(columns=culex_rename,inplace=True)

extra_train_col = [col for col in final_train.columns if col not in set(final_test.columns)]
print(f' extra train columns are {extra_train_col},  \nshape of final train set is {final_train.shape}')

extra_test_col = [col for col in final_test.columns if col not in set(final_train.columns)]
print(f' extra test columns are {extra_test_col}, \nshape of final test set is {final_test.shape}')


 extra train columns are ['WnvPresent', 'NumMosquitos'],  
shape of final train set is (8610, 23)
 extra test columns are ['Id'], 
shape of final test set is (116293, 22)


<a id='merge'></a>

### Merging Weather into Train & Test

In [21]:
#importing the cleaned weather csv from the 'data' folder
weather = pd.read_csv('../data/df_weather_clean.csv')

In [22]:
#setting the date column to datetime and assign it to the index
weather['Date'] = pd.to_datetime(weather['Date'])
weather.set_index('Date',inplace=True)

#merging the weather data to each row based on the station assigned to it
station1 = weather[weather['Station']==1]
station2 = weather[weather['Station']==2]

In [23]:
# merging weather into train. each trap will be assigned appropriate data from the closest weather station
final_train.set_index('Date',inplace=True)

final_train1=final_train[final_train['station']==1]
final_train2=final_train[final_train['station']==2]

final1 = pd.merge(final_train1,station1,left_index=True, right_index=True,how='left')
final2 = pd.merge(final_train2,station2,left_index=True, right_index=True,how='left')

weather_train_merged = pd.concat([final1,final2],axis=0)

In [24]:
# merging weather into test
final_test.set_index('Date',inplace=True)

final_test1=final_test[final_test['station']==1]
final_test2=final_test[final_test['station']==2]

finaltest1 = pd.merge(final_test1,station1,left_index=True, right_index=True,how='left')
finaltest2 = pd.merge(final_test2,station2,left_index=True, right_index=True,how='left')

weather_test_merged = pd.concat([finaltest1,finaltest2],axis=0)

In [25]:
weather_train_merged.drop(columns = ['week_x', 'year_x'], inplace = True)
weather_test_merged.drop(columns = ['week_x', 'year_x'], inplace = True)

In [26]:
weather_test_merged.head()

,Id,Trap,Latitude,Longitude,year_month,station,carrier,jul,jun,oct,sep,species_pipiens,species_pipiens/restuans,species_restuans,species_salinarius,species_tarsalis,species_territans,species_unspecified_culex,may,Station,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,year_y,day,month,drizzle,smoke,storm,snow,mist,rain,h_fog,fog,haze,week_y,tavg_lag_1,tavg_lag_3,tavg_lag_7,preciptotal_lag_1,preciptotal_lag_3,preciptotal_lag_7,sunrise_clean,sunset_clean,DaylightHrs,DaylightHrs_lag_1,DaylightHrs_lag_3,DaylightHrs_lag_7,relhum,relhum_lag_1,relhum_lag_3,relhum_lag_7
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2008-06-11,1,T002,41.95469,-87.800991,"(2008, 6)",1,1,0,1,0,0,0,1,0,0,0,0,0,0.0,1,86,61,74,56,64,0,9,416,1926,0.0,0.0,29.28,29.99,8.9,18,10.0,2008,11,Jun,0,0,0,0,0,0,0,0,0,24,70.0,70.0,78.0,0.01,0.21,0.07,1900-01-01 04:16:00,1900-01-01 19:26:00,15,15.0,15.0,15.0,53.449175,61.176087,81.306673,79.152204
2008-06-11,2,T002,41.95469,-87.800991,"(2008, 6)",1,1,0,1,0,0,0,0,1,0,0,0,0,0.0,1,86,61,74,56,64,0,9,416,1926,0.0,0.0,29.28,29.99,8.9,18,10.0,2008,11,Jun,0,0,0,0,0,0,0,0,0,24,70.0,70.0,78.0,0.01,0.21,0.07,1900-01-01 04:16:00,1900-01-01 19:26:00,15,15.0,15.0,15.0,53.449175,61.176087,81.306673,79.152204
2008-06-11,3,T002,41.95469,-87.800991,"(2008, 6)",1,1,0,1,0,0,1,0,0,0,0,0,0,0.0,1,86,61,74,56,64,0,9,416,1926,0.0,0.0,29.28,29.99,8.9,18,10.0,2008,11,Jun,0,0,0,0,0,0,0,0,0,24,70.0,70.0,78.0,0.01,0.21,0.07,1900-01-01 04:16:00,1900-01-01 19:26:00,15,15.0,15.0,15.0,53.449175,61.176087,81.306673,79.152204
2008-06-11,4,T002,41.95469,-87.800991,"(2008, 6)",1,0,0,1,0,0,0,0,0,1,0,0,0,0.0,1,86,61,74,56,64,0,9,416,1926,0.0,0.0,29.28,29.99,8.9,18,10.0,2008,11,Jun,0,0,0,0,0,0,0,0,0,24,70.0,70.0,78.0,0.01,0.21,0.07,1900-01-01 04:16:00,1900-01-01 19:26:00,15,15.0,15.0,15.0,53.449175,61.176087,81.306673,79.152204
2008-06-11,5,T002,41.95469,-87.800991,"(2008, 6)",1,0,0,1,0,0,0,0,0,0,0,1,0,0.0,1,86,61,74,56,64,0,9,416,1926,0.0,0.0,29.28,29.99,8.9,18,10.0,2008,11,Jun,0,0,0,0,0,0,0,0,0,24,70.0,70.0,78.0,0.01,0.21,0.07,1900-01-01 04:16:00,1900-01-01 19:26:00,15,15.0,15.0,15.0,53.449175,61.176087,81.306673,79.152204


In [27]:
# checking the extra columns in the merged test weather and merged train weather
extra_train_col = [col for col in weather_train_merged.columns if col not in set(weather_test_merged.columns)]
print(f' extra train columns are {extra_train_col},  \nshape of merged weather and train set is {weather_train_merged.shape}')

extra_test_col = [col for col in weather_test_merged.columns if col not in set(weather_train_merged.columns)]
print(f' extra test columns are {extra_test_col}, \nshape of merged weather and test set is {weather_test_merged.shape}')

 extra train columns are ['WnvPresent', 'NumMosquitos'],  
shape of merged weather and train set is (8610, 66)
 extra test columns are ['Id'], 
shape of merged weather and test set is (116293, 65)


In [28]:
# splitting train into train and validation set. Validation set: 2013 records, train set: 2007, 2009, 2011
train2013 = weather_train_merged['2013']
train2011 = weather_train_merged['2011']
train2009 = weather_train_merged['2009']
train2007 = weather_train_merged['2007']

In [29]:
train_0709 = train2007.append(train2009)
train_070911 = train_0709.append(train2011)

In [30]:
pd.DataFrame(train_070911).to_csv('../data/train_070911.csv')
pd.DataFrame(train2013).to_csv('../data/train2013.csv')
pd.DataFrame(weather_test_merged).to_csv('../data/weather_test_merged.csv')
